# Make an interactive map of precipitation

In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import os
import glob
import xarray as xr
import folium as fm
import rioxarray as rxr
import numpy.ma as ma
import matplotlib.pyplot as plt
# import branca
# import branca.colormap as cm
from matplotlib import cm
import hvplot.pandas
import geopandas as gpd
import branca

### Load sample data

In [2]:
# define data file name
data_path = '/Users/raineyaberle/Courses/GEOS_505_ResearchComputing/data/CFS_Sample_Series.nc'

# load data as xarray.Dataset
data = xr.open_dataset(data_path)
data

<xarray.Dataset>
Dimensions:     (time: 5, latitude: 190, longitude: 384)
Coordinates:
  * time        (time) datetime64[ns] 2018-10-31 2018-10-31 ... 2018-10-31
    step        (time) timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 89.28 88.34 87.4 ... -87.4 -88.34 -89.28
  * longitude   (longitude) float64 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
    valid_time  (time) datetime64[ns] ...
Data variables: (12/53)
    uflx        (time, latitude, longitude) float32 ...
    vflx        (time, latitude, longitude) float32 ...
    shtfl       (time, latitude, longitude) float32 ...
    lhtfl       (time, latitude, longitude) float32 ...
    t           (time, latitude, longitude) float32 ...
    sdwe        (time, latitude, longitude) float32 ...
    ...          ...
    snohf       (time, latitude, longitude) float32 ...
    vbdsf       (time, latitude, longitude) float32 ...
    vddsf       (time, latitude, longitude) float32 ...
    nbdsf       (time, latitude, longitude) float32 ...
    nddsf       (time, latitude, longitude) float32 ...
    srweq       (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2022-11-14T11:21 GRIB to CDM+CF via cfgrib-0.9.1...

### 0. Some minor data preprocessing

In [3]:
# grab a spatial subset for the first time slice
data_PNW = data.where((data.latitude > 35) & (data.latitude < 50)
                    & (data.longitude > 230) & (data.longitude < 250), drop=True)
data_PNW_ti = data_PNW.isel(time=0)

### 1. Create map of air temperature in 2018 

In [4]:
import matplotlib

# create map
m = fm.Map(location = [40, 240], zoom_start = 4, tiles='StamenTerrain')

# get extent
xmin, xmax = np.min(data_PNW_ti.longitude.data), np.max(data_PNW_ti.longitude.data)
ymin, ymax = np.min(data_PNW_ti.latitude.data), np.max(data_PNW_ti.latitude.data)

# create cmap
top = matplotlib.cm.get_cmap('Oranges_r', 128)
bottom = matplotlib.cm.get_cmap('Blues', 128)
newcolors = np.vstack((top(np.linspace(0, 1, 128)),
                       bottom(np.linspace(0, 1, 128))))
cmap = matplotlib.colors.ListedColormap(newcolors, name='OrangeBlue')
# create colormap for legend
cmap_legend = branca.colormap.LinearColormap([cmap(0), cmap(56), cmap(128), cmap(184), cmap(256)], 
                                      index=[250, 265, 280, 295, 310], vmin=250, vmax=310, 
                                      caption='Temperature [K]', max_labels=10, tick_labels=None)

# "colorize" the data
def colorize(array, cmap=cmap):
    normed_data = (array - array.min()) / (array.max() - array.min())    
    cm = cmap    
    return cm(normed_data)


# add image to map
data_colorized = colorize(data_PNW_ti['t'].data)
fm.raster_layers.ImageOverlay(image=data_colorized, 
                              bounds=[[ymin, xmin], [ymax, xmax]], 
                              opacity=0.8,
                              origin='upper', 
                            ).add_to(m)
m.add_child(cmap_legend)
m